In [ ]:
import os, sys
import warnings
from IPython.core.display import HTML
from IPython.display import Image
from owslib.wms import WebMapService 
from owslib.util import Authentication
from skimage import io
from datetime import datetime, time, timedelta, date
import requests
import os.path

sys.path.append(os.path.dirname(os.getcwd()))
import authorisation_functions as auth

# Deaktivierung von Warnungen
warnings.simplefilter("ignore")
service_url = 'https://view.eumetsat.int/geoserver/ows?'
wms = WebMapService(service_url, auth=Authentication(verify=False))

target_layer = 'msg_fes:rgb_dust'
format_option = 'image/png'

# Karten für einen Bereich von Daten

payload = {
'layers' : [target_layer + ",backgrounds:ne_10m_coastline"], # mit Küstenlinie
'styles' : '',
'format' : format_option,
'crs'    : 'EPSG:4326',
'bbox'   : (-20, 7, 52, 37), # order is lat1,lon1,lat2,lon2
'size'   : (168,70),
'time' : time
}

start_time = datetime.now()

# Erstelle Liste von Zeitstempeln ausgehend von start_time und gehe numb * timedelta (später definiert) Schritte zurück
numb = int(input("Wieviele Zeitstempel?"))
dateList = []
count = 0

date_list = [start_time - timedelta(minutes = x * 15) for x in range(numb)]

for cur_time in  date_list:
    
    # Bestimme Zeitintervall im geforderten Format
    time_begin = cur_time.strftime('%Y-%m-%dT%H:%M:%S.%f')[:-3]
    time_end = (cur_time + timedelta(hours=0, minutes=15)).strftime('%Y-%m-%dT%H:%M:%S.%f')[:-3]
    time = time_begin + 'Z' + '/' + time_end + 'Z'
    #print(time)
    payload['time'] = time
    
    # Laden des Bildes
    try:
        wms = WebMapService(service_url, auth=Authentication(verify=False))
        img = wms.getmap(**payload)
        Image(img.read())

        # Speichern des Bildes
        save_path = os.getcwd() + '/pictures/pngs'
        time_file = cur_time.strftime('%Y-%m-%d__%H--%M--%S')
        time_file = time_file + '.png'
        completeName = os.path.join(save_path, time_file)
        out = open(completeName, 'wb')

        out.write(img.read())
        out.close()
        
        # Zählen der gespeicherten Bilder
        count += 1
        if((count % 50) == 0):
            print(f"Die ersten {count} Bilder wurden erfolgreich heruntergeladen!")
    
    except Exception:   
        print(f"Zeitstempel {time_begin} wurde übersprungen, da kein Bild zum Download zur Verfügung ist.")
        
print(f"Insgesamt wurden {count} Bilder heruntergeladen!")